In [317]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import calmap
import folium
import datetime as dt

In [318]:
def update_and_clean():
    corona=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    go_corona=cleaning(corona)
    df_death=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
    df_death=cleaning(df_death)
#     recovered=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')
#     recovered=cleaning(recovered)
    return go_corona,df_death

In [319]:
def cleaning(dataset):
    dataset.replace(0, np.NAN, inplace=True)
    go_corona=pd.melt(dataset,id_vars=['Province/State','Country/Region','Lat','Long'],value_name='Cases',var_name='Date')
    go_corona['Date'] = pd.to_datetime(go_corona['Date'])
    go_corona['Date'] = go_corona['Date'].dt.strftime('%m/%d/%Y')
    go_corona.dropna(axis=0,subset=['Cases'],inplace=True)
    go_corona.reset_index(drop=True,inplace=True)
    go_corona[go_corona['Cases']<0]=0
    return go_corona

In [328]:
def plot_world_map(go_corona,title='Corona'):
    fig = px.scatter_geo(go_corona,lon=go_corona['Long'], lat=go_corona['Lat'], locationmode='country names', 
                         color="Cases",hover_name="Country/Region",size=go_corona['Cases']**0.3,
                         hover_data=[go_corona['Country/Region'],go_corona['Province/State'],go_corona['Cases']],
                         range_color= [0,max(go_corona['Cases'])], color_continuous_scale=["blue", "green", "red"],
                         projection="natural earth", animation_frame="Date", 
                         title=title+' Cases over time')
    fig.update(layout_coloraxis_showscale=True)
    fig.show()

In [322]:
def plot_curve(Conf,Death,Reco=None,country=None,compare=False,country2=None):
    if country != None:
        Conf1=group_by_(Conf,col=['Date','Country/Region'])
        Death1=group_by_(Death,col=['Date','Country/Region'])
        #Reco1=group_by_(Reco,col=['Date','Country/Region'])

        Conf1=Conf1[Conf1['Country/Region']==country]
        Death1=Death1[Death1['Country/Region']==country]
        #Reco1=Reco1[Reco1['Country/Region']==country]
        
        Conf1.reset_index(drop=True,inplace=True)
        Death1.reset_index(drop=True,inplace=True)
        #Reco1.reset_index(drop=True,inplace=True)
    else:
        Conf1=group_by_(Conf)
        Death1=group_by_(Death)
        #Reco1=group_by_(Reco)
    fig = go.Figure()
    fig.update_layout(template='plotly_dark')
    fig.add_trace(go.Scatter(x=Conf1['Date'], 
                             y=Conf1['Cases'],
                             mode='lines+markers',
                             name='Confirmed',
                             line=dict(color='Yellow', width=2)
                            ))
    fig.add_trace(go.Scatter(x=Death1['Date'],
                             y=Death1['Cases'],
                             mode='lines+markers',
                             name='Deaths',
                             line=dict(color='Red', width=2)
                            ))
#     fig.add_trace(go.Scatter(x=Reco1['Date'], 
#                              y=Reco1['Cases'],
#                              mode='lines+markers',
#                              name='Recoveries',
#                              line=dict(color='Green', width=2)
#                             ))

    if country2 != None:
        Conf2=group_by_(Conf,col=['Date','Country/Region'])
        Death2=group_by_(Death,col=['Date','Country/Region'])
#         Reco2=group_by_(Reco,col=['Date','Country/Region'])

        Conf2=Conf2[Conf2['Country/Region']==country2]
        Death2=Death2[Death2['Country/Region']==country2]
#         Reco2=Reco2[Reco2['Country/Region']==country2]
        
        Conf2.reset_index(drop=True,inplace=True)
        Death2.reset_index(drop=True,inplace=True)
#         Reco2.reset_index(drop=True,inplace=True)
        
        compare=True
    elif country!=None:
        Conf2=group_by_(Conf)
        Death2=group_by_(Death)
#         Reco2=group_by_(Reco)
        country2='World'
    if compare:
            fig.add_trace(go.Scatter(x=Conf2['Date'], 
                             y=Conf2['Cases'],
                             mode='markers',
                             name='Confirmed-'+country2,
                             line=dict(color='Blue', width=1)
                            ))
            fig.add_trace(go.Scatter(x=Death2['Date'],
                                     y=Death2['Cases'],
                                     mode='markers',
                                     name='Deaths-'+country2,
                                     line=dict(color='Orange', width=1)
                                    ))
#             fig.add_trace(go.Scatter(x=Reco2['Date'], 
#                                      y=Reco2['Cases'],
#                                      mode='markers',
#                                      name='Recoveries-'+country2,
#                                      line=dict(color='Violet', width=1)
#                                     ))

    fig.show()
    

In [323]:
def group_by_(dataset,col=['Date']):
    grouped_df = dataset.groupby(col).sum().reset_index()
    return grouped_df

In [324]:
go_corona,df_death=update_and_clean()

In [329]:
plot_world_map(go_corona,'Confirmed')

In [330]:
plot_world_map(df_death,'Death')

In [ ]:
# plot_world_map(recovered,'Recovered')

In [331]:
plot_curve(go_corona,df_death,country='US',country2='China')